# Практическая работа №1

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

from sklearn.metrics import mean_squared_error, r2_score

from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline

### Загрузим данные для работы. 

In [2]:
df = pd.read_csv("Elections.csv",delimiter=';')
df.head()

,Region Index,Subjects of Russia,Unemployment rate. %,Income per capita. RUB,Average salary. RUB per month,Spendings per capita. RUB,State housing. %,Higher Education. %,Number of doctors per 10000 citizens,GRP per capita. RUB,Investment in total capital,Consumer Price Index (CPI),Average income / min. living wage,Average costs / min. living wage,Votes for United Russia. %
0,1,Adygea,43505.0,12236,12787.4,8361,0.6,4.0,37.9,93548.3,25834,110.1,42036,17168.0,61.002
1,2,Altai,43685.0,10926,12050.7,8139,43680.0,43694.0,48.7,107282.4,23048,108.2,33604,15707.0,37.174
2,3,Amur Oblast,43714.0,14064,21207.5,9406,2.0,43590.0,62.9,175927.8,95619,109.4,17199,23743.0,43.543
3,4,Arkhangelsk Oblast,43503.0,19310,22192.1,11688,43527.0,9.0,56.5,256728.4,63912,109.4,14305,43587.0,31.900
4,5,Astrakhan Oblast,43532.0,14641,16582.1,10651,43586.0,43472.0,68.2,131400.1,56338,109.8,20852,31778.0,60.170


Будем решать задачу регрессии: необходимо предсказать процент голосов за Единую Россию в каждом регионе по различным характеристикам уровня жизни в регионе.

## Шаг 1. 
Создайте матрицу X объект-признак и целевой вектор y ("Votes for United Russia. %")

In [14]:
y = df['Votes for United Russia. %']
X = df.drop(['Votes for United Russia. %'],axis=1)

## Шаг 2.
Оставьте в матрице X только столбцы 'Unemployment rate. %', 'Spendings per capita. RUB', 'Number of doctors per 10000 citizens', 'Average salary. RUB per month'

In [15]:
X = X[['Unemployment rate. %', 'Spendings per capita. RUB',
       'Number of doctors per 10000 citizens',
        'Average salary. RUB per month']]

## Шаг 3.
Разбейте данные на train и test (доля тестовых данных - 30%).

In [5]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.3)

## Шаг 4.
Обучите линейную регрессию на тренировочных данных и сделайте предсказания на train и на test.

In [6]:
model = LinearRegression()
model.fit(Xtrain, ytrain)

pred_train = model.predict(Xtrain)
pred_test = model.predict(Xtest)

## Шаг 5.
Выведите на экран ошибку MSE на train и на test, затем выведите на экран ошибку r2 на train и test.  

In [7]:
print(mean_squared_error(ytrain, pred_train), mean_squared_error(ytest, pred_test))
print(r2_score(ytrain, pred_train), r2_score(ytest, pred_test))

183.84900042994443 437.6610266845584
0.22766620061822485 -0.2344327448250607


## Шаг 6.
Вычислите среднее качество (r2) модели на кросс-валидации с k=5 фолдами.

In [16]:
cross_val_score(model, X, y, cv=5, scoring='r2').mean()

-0.2787293912365831

## Шаг 7.
Теперь примените линейную регрессию с L1-регуляризацией (Lasso) для данной задачи. Объявите модель и подберите параметр регуляризации alpha по сетке. Ищите alpha в диапазоне (0.1, 1.1) с шагом 0.1. 

Осуществите подбор параметра alpha по тренировочным данным (Xtrain, ytrain).

In [18]:
from sklearn.linear_model import Lasso

model = Lasso()

alpha_grid = {'alpha': np.arange(0.1, 1.1, 0.1)}

grid = GridSearchCV(model, alpha_grid, cv=3, n_jobs=-1)
grid.fit(Xtrain, ytrain)

C:\Users\elena\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'alpha': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

## Шаг 8.
Выведите наилучший алгоритм и наилучшее качество по результатам подбора alpha (best_estimator_ и best_score_).

In [10]:
grid.best_estimator_, grid.best_score_

(Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
    normalize=False, positive=False, precompute=False, random_state=None,
    selection='cyclic', tol=0.0001, warm_start=False), -5293.801990764731)

## Шаг 9.

С помощью найденного best_estimator_ сделайте предсказание на тестовых данных и выведите на экран r2-score на тесте.

In [11]:
pred_test = grid.best_estimator_.predict(Xtest)

r2_score(ytest, pred_test)

-0.23443655294807675

## Шаг 10.

Попробуем улучшить качество модели за счет добавления полиномиальных признаков. Создайте pipeline, состоящий из добавления полиномиальных признаков степени 2, а затем применения линейной регрессии.

Затем вычислите r2-score этой модели на кросс валидации с пятью фолдами.

In [19]:
pipe = Pipeline([('poly',PolynomialFeatures(degree=2)), \
                 ('model',LinearRegression())])

cross_val_score(pipe, X, y, cv=3, scoring='r2').mean()

-407788.0331865719

## Шаг 11.
Обучите модель (pipeline) на тренировочных данных и сделайте предсказания для train и test, затем выведите на экран r2-score и MSE на тренировочных и на тестовых данных.

In [13]:
pipe.fit(Xtrain, ytrain)

pred_tr = pipe.predict(Xtrain)
pred_te = pipe.predict(Xtest)

print(r2_score(ytrain, pred_tr), r2_score(ytest, pred_te))
print(mean_squared_error(ytrain, pred_tr), mean_squared_error(ytest, pred_te))

0.5225650415744894 -99929843.48012334
113.6502635869408 35429551358.666405


## Сделайте выводы. Для этого ответьте на вопросы:

1) Хорошее ли качество показала исходная модель (линейная регрессия без регуляризации)? Является ли эта модель переобученной?

2) Помогла ли L1-регуляризация улучшить качество модели?

3) Помогло ли добавление полиномов второй степени улучшить качество модели? Как добавление новых признаков повлияло на переобучение?